In [ ]:
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime

import math

In [ ]:
#add the access token you got from NOAA
Token = 'JvgZzITnvipZbRytfbTgJVeCmCvLwhWu'

#Long Beach Airport station
station_id = 'GHCND:USW00023129'

In [ ]:
baseurl = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/locations'
params_diction = {}
# params_diction['datasetid'] = 'GHCND'
# params_diction['datatypeid'] = 'TAVG'
params_diction['limit'] = 1000
# params_diction['locationcategoryid'] = 'ST'
# params_diction['stationid'] = 'GHCND:USW00023129'
# params_diction['startdate'] = '2020-01-01'
# params_diction['enddate'] = '2021-05-07'
params_diction['offset'] = 1000
response = requests.request('GET', baseurl, params=params_diction, headers={'token':Token})

In [ ]:
response.json()['metadata']

In [ ]:
total_location_count = response.json()['metadata']['resultset']['count']
total_location_count

In [ ]:
# #initialize dataframe
# df_temp = pd.DataFrame()

# #populate date and average temperature fields (cast string date to datetime and convert temperature from tenths of Celsius to Fahrenheit)
# df_temp['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
# df_temp['avgTemp'] = [float(v)/10.0*1.8 + 32 for v in temps]

# Get all locationid

In [ ]:
baseurl = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/locations'
all_locations_dict = {}
for offset in np.arange(0, 38862, 1000): # can use total_location_count to avoid hard coding
    params_diction = {}
    params_diction['offset'] = offset
    params_diction['limit'] = 1000
    response = requests.request('GET', baseurl, params=params_diction, headers={'token':Token})
    current_dict = json.loads(response.text)
    for item in current_dict['results']:
        all_locations_dict[item['id']] = item['name']

In [ ]:
len(all_locations_dict)

In [ ]:
df = pd.DataFrame.from_dict([all_locations_dict])
df = pd.melt(df)

In [ ]:
df.to_csv('all_locations.csv')

# Generate the list of all locations and remove ones that returns error \[400]
## It's a manual and iterative process...

In [ ]:
mylist = ['01', '02', '03', '04', '05', '06', '07', '08', '09']
from_ten = list(np.arange(10, 80, 1))
from_ten = [str(x) for x in from_ten]
mylist.extend(from_ten)
mylist = [str(x) for x in mylist]
len(mylist)

In [ ]:
# Could turn into a for loop here...
mylist.remove('03')
mylist.remove('07')
mylist.remove('11')
mylist.remove('14')
mylist.remove('43')
mylist.remove('52')
mylist.remove('57')
mylist.remove('58')
mylist.remove('59')
mylist.remove('61')
mylist.remove('62')
mylist.remove('63')
mylist.remove('64')
mylist.remove('65')
mylist.remove('67')
mylist.remove('68')
mylist.remove('69')
mylist.remove('70')
mylist.remove('71')
mylist.remove('72')
mylist.remove('73')
mylist.remove('74')
mylist.remove('75')
mylist.remove('76')
mylist.remove('77')
mylist.remove('78')

len(mylist)

# For Year 2020
## Get number of records per each locationid

In [ ]:
baseurl = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data'
records_dict = {}
for n in mylist:
    locationid = f'FIPS:{n}'
    print(locationid)
    params_diction = {}
    params_diction['datasetid'] = 'GHCND'
    params_diction['limit'] = 1000
    params_diction['datatypeid'] = 'TAVG'

    params_diction['locationid']=locationid

    params_diction['startdate'] = '2020-01-01'
    params_diction['enddate'] = '2020-12-31'
    response = requests.request('GET', baseurl, params=params_diction, headers={'token':Token})
    current_dict = json.loads(response.text)
    number_of_records = current_dict['metadata']['resultset']['count']
    records_dict[locationid] = number_of_records

In [ ]:
df_records_dict = pd.DataFrame.from_dict([records_dict])
df_records_dict = pd.melt(df_records_dict)
len(df_records_dict)

In [ ]:
df_records_dict.to_csv('records_per_loc.csv')

## Add State name to df

In [ ]:
df.columns = ['locationid', 'location']
df.head()

In [ ]:
df_records_dict.columns = ['locationid', 'location']
df_records_dict.head()

In [ ]:
location_df = pd.merge(df_records_dict, df, how='inner', on='locationid')

In [ ]:
location_df.head(20)

In [ ]:
location_df.columns = ['locationid', 'records_count2020', 'states']
location_df.head()

### Get the total number of requests needed.
### Turns out 879 requests needed in total

In [ ]:
location_df['requests_needed'] = (location_df['records_count2020'] / 1000).apply(math.ceil)

In [ ]:
location_df['requests_needed'].sum()

## Get data for 2020
### The limit for daily data is 1 year, so has to split into 2020 and 2021

In [ ]:
# get data for 2020

baseurl = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data'

request_counter = 0

for i in range(len(location_df)):
    state = location_df.iloc[i]['states']
    records_count = location_df.iloc[i]['records_count2020']
    locationid = location_df.iloc[i]['locationid']

    for offset in np.arange(1, records_count, 1000):
        params_diction = {}
        params_diction['datasetid'] = 'GHCND'
        params_diction['limit'] = 1000
        params_diction['datatypeid'] = 'TAVG'
        params_diction['locationid']=locationid
        params_diction['offset'] = offset
        params_diction['startdate'] = '2020-01-01'
        params_diction['enddate'] = '2020-12-31'

        response = requests.request('GET', baseurl, params=params_diction, headers={'token':Token})

        try:
            current_dict = json.loads(response.text)
            if request_counter==0:
                current_ouput = pd.DataFrame(current_dict['results'])
                current_ouput['locationid'] = locationid
                current_ouput['state'] = state
                df_output = current_ouput
                print('now running, successed on 1st request')
            else:
                current_ouput = pd.DataFrame(current_dict['results'])
                current_ouput['locationid'] = locationid
                current_ouput['state'] = state
                df_output = df_output.append(current_ouput)

        except:
            print(state, offset)

        request_counter+=1

        if request_counter % 50 == 0:
            print(request_counter)

In [ ]:
current_dict['metadata']

In [ ]:
len(df_output)

In [ ]:
df_output.head()

In [ ]:
df_output.tail()

In [ ]:
df_output.to_csv('output20v2.gzip', compression='gzip')
# the line below is for use in Colab
# !cp output20v2.gzip "drive/MyDrive"

# For Year 2021
## Get number of records for each location

In [ ]:
baseurl = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data'
records_dict21 = {}
for n in mylist:
    locationid = f'FIPS:{n}'
    params_diction = {}
    params_diction['datasetid'] = 'GHCND'
    params_diction['limit'] = 1000
    params_diction['datatypeid'] = 'TAVG'
    params_diction['locationid']=locationid
    params_diction['startdate'] = '2020-01-01'
    params_diction['enddate'] = '2020-12-31'
    response = requests.request('GET', baseurl, params=params_diction, headers={'token':Token})
    current_dict = json.loads(response.text)
    number_of_records = current_dict['metadata']['resultset']['count']
    records_dict21[locationid] = number_of_records

In [ ]:
df_records_dict21 = pd.DataFrame.from_dict([records_dict21])
df_records_dict21 = pd.melt(df_records_dict21)
len(df_records_dict21)

In [ ]:
df_records_dict21.head()

# Add State name to df

In [ ]:
df_records_dict21.columns = ['locationid', 'location']
df_records_dict21.head()

In [ ]:
location_df21 = pd.merge(df_records_dict21, df, how='inner', on='locationid')
location_df21.head()

In [ ]:
location_df21.columns = ['locationid', 'records_count2021', 'states']
location_df21.head()

In [ ]:
location_df21['requests_needed'] = (location_df21['records_count2021'] / 1000).apply(math.ceil)

### Only 306 requests this time

In [ ]:
location_df21['requests_needed'].sum()

## Get data for 2021

In [ ]:
# # get data for 2021, Version 1, locally in VS Code
# # This wasn't actually tested as I used the 2nd version on Colab
# # which doesn't need the "try" again thing.
# # I think it's just Googles's connection is much better than mine

# baseurl = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data'

# df_output21 = df_output.copy()

# request_counter = 0

# for i in range(len(location_df)):
#     state = location_df21.iloc[i]['states']
#     records_count = location_df21.iloc[i]['records_count2020']
#     locationid = location_df21.iloc[i]['locationid']



#     for offset in np.arange(1, records_count, 1000):
        
#         params_diction = {}
#         params_diction['datasetid'] = 'GHCND'
#         params_diction['limit'] = 1000
#         params_diction['datatypeid'] = 'TAVG'
#         params_diction['locationid']=locationid
#         params_diction['offset'] = offset
#         params_diction['startdate'] = '2021-01-01'
#         params_diction['enddate'] = '2021-04-30'

#         response = requests.request('GET', baseurl, params=params_diction, headers={'token':Token})
        
#         try:
#             current_dict = json.loads(response.text)
#             if request_counter==0:
#                 df_output = pd.DataFrame(current_dict['results'])
#                 print('now running')
#             else:
#                 df_output = df_output.append(pd.DataFrame(current_dict['results']))

#         except:
#             try:
#                 print('Try again on:', state, offset)
#                 response = requests.request('GET', baseurl, params=params_diction, headers={'token':Token})
#                 current_dict = json.loads(response.text)
#                 df_output = df_output.append(pd.DataFrame(current_dict['results']))
#             except:
#                 print('Failed on:', state, offset)

#         if request_counter % 50 ==0:
#             print(request_counter)

In [ ]:
# get data for 2021, Version 2, was run successfully on Colab

baseurl = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data'

request_counter = 0

for i in range(len(location_df21)):
    state = location_df21.iloc[i]['states']
    records_count = location_df21.iloc[i]['records_count2021']
    locationid = location_df21.iloc[i]['locationid']

    for offset in np.arange(1, records_count, 1000):
        
        params_diction = {}
        params_diction['datasetid'] = 'GHCND'
        params_diction['limit'] = 1000
        params_diction['datatypeid'] = 'TAVG'
        params_diction['locationid']=locationid
        params_diction['offset'] = offset
        params_diction['startdate'] = '2021-01-01'
        params_diction['enddate'] = '2021-04-30'

        response = requests.request('GET', baseurl, params=params_diction, headers={'token':Token})
        
        try:
            current_dict = json.loads(response.text)
            if request_counter==0:
                current_ouput = pd.DataFrame(current_dict['results'])
                current_ouput['locationid'] = locationid
                current_ouput['state'] = state
                df_output21 = current_ouput
                print('now running, successed on 1st request')
            else:
                current_ouput = pd.DataFrame(current_dict['results'])
                current_ouput['locationid'] = locationid
                current_ouput['state'] = state
                df_output21 = df_output21.append(current_ouput)

        except:
            print(state, offset)

        request_counter+=1

        if request_counter % 50 == 0:
            print(request_counter)

In [ ]:
df_output21.to_csv('output21.gzip', compression='gzip')
# the line below is for use in Colab
# !cp output21.gzip "drive/MyDrive"